In [8]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import seaborn as sns
from scipy.stats.mstats import winsorize
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('select * from useducation',con=engine)

engine.dispose()

columns = list(df.columns)
remove = {'PRIMARY_KEY', 'STATE', 'YEAR', 'AVG_READING_8_SCORE', 'AVG_READING_4_SCORE', 'AVG_MATH_8_SCORE', 'AVG_MATH_4_SCORE'}
columns = [e for e in columns if e not in remove]

states = df['STATE'].unique()

for state in states:
    df.loc[df['STATE'] == state, columns] = df.loc[df['STATE'] == state, columns].interpolate()

df.dropna(inplace=True)

print(df.head())

         PRIMARY_KEY       STATE  YEAR     ENROLL  TOTAL_REVENUE  \
209     1996_ALABAMA     ALABAMA  1996   735912.0      3365113.0   
211     1996_ARIZONA     ARIZONA  1996   764681.0      4143304.0   
212    1996_ARKANSAS    ARKANSAS  1996   452907.0      2183384.0   
213  1996_CALIFORNIA  CALIFORNIA  1996  5460484.0     31282981.0   
214    1996_COLORADO    COLORADO  1996   655679.0      3891203.0   

     FEDERAL_REVENUE  STATE_REVENUE  LOCAL_REVENUE  TOTAL_EXPENDITURE  \
209         334858.0      2138717.0       891538.0          3572283.0   
211         378917.0      1798363.0      1966024.0          4391555.0   
212         192152.0      1296247.0       694985.0          2297381.0   
213        2603882.0     17061474.0     11617625.0         31561692.0   
214         194998.0      1664676.0      2031529.0          4075165.0   

     INSTRUCTION_EXPENDITURE  ...  GRADES_4_G  GRADES_8_G  GRADES_12_G  \
209                1987018.0  ...     57098.0     58305.0      42510.0   
211 

1. Create a new score variable from the weighted averages of all score variables in the datasets. 

In [11]:
df['avg_score'] = (df['GRADES_4_G']*((df['AVG_MATH_4_SCORE']+df['AVG_READING_4_SCORE'])/2)+
                   df['GRADES_8_G']*((df['AVG_MATH_8_SCORE']+df['AVG_READING_8_SCORE'])/2))/(df['GRADES_8_G']+df['GRADES_4_G'])

2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [14]:
df[["avg_score", "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE",
              "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()
print('Inatruction expenditure is the most correlated')

Inatruction expenditure is the most correlated


3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [17]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df[['INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE',
                  'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
df["pca_1"] = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.9430175]


4. What is the correlation between the overall score variable and the 1st principal component?

In [20]:
df[['avg_score', 'pca_1', 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
              'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']].corr()

,avg_score,pca_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
avg_score,1.000000,0.044596,0.074155,0.087386,0.079002,0.010959,-0.005389
pca_1,0.044596,1.000000,0.992054,0.968606,0.988345,0.971343,0.955792
TOTAL_EXPENDITURE,0.074155,0.992054,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.087386,0.968606,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.988345,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.971343,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.955792,0.925784,0.884798,0.911245,0.918188,1.000000


The correlation the first principal component is less than the correlation for both instruction and support service expenditures